# Quran Verse Detection - Surah An-Naba

## Dataset Structure
Dataset ini berisi rekaman audio Surah An-Naba dari 7 pembaca Quran yang berbeda:

### Folder Structure:
```
quran_detect/
├── sample_1/     # Pembaca 1
├── sample_2/     # Pembaca 2
├── sample_3/     # Pembaca 3
├── sample_4/     # Pembaca 4
├── sample_5/     # Pembaca 5
├── sample_6/     # Pembaca 6
└── sample_7/     # Pembaca 7
```

### File Naming Convention:
Setiap folder berisi 41 file audio dengan format:
- `078000.mp3` → **Bismillah** (Pembuka) → **Label 0**
- `078001.mp3` → **Ayat 1** → **Label 1**
- `078002.mp3` → **Ayat 2** → **Label 2**
- ...
- `078040.mp3` → **Ayat 40** → **Label 40**

### Total Dataset:
- **7 pembaca** × **41 ayat** = **287 audio samples**
- **41 kelas** (0-40): 1 Bismillah + 40 Ayat

## Tujuan Model:
Model dilatih untuk mendeteksi ayat mana yang sedang dibacakan ketika diberi audio testing baru.

In [1]:
# ======= IMPORT LIBRARIES =======

import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
import pandas as pd
from tqdm import tqdm
import pickle
import json
import scipy.signal
import plotly.graph_objects as go

print("✅ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Librosa version: {librosa.__version__}")

✅ All libraries imported successfully!
TensorFlow version: 2.18.1
Librosa version: 0.10.2.post1


In [2]:
# ======= ADVANCED AUDIO PREPROCESSING =======

def extract_advanced_features(file_path, max_length=256, sr=22050):
    """
    Extract comprehensive audio features for better verse detection
    """
    try:
        # Load audio dengan preprocessing
        audio, sample_rate = librosa.load(file_path, sr=sr)
        
        # 1. Audio Preprocessing
        audio = librosa.util.normalize(audio)
        audio, _ = librosa.effects.trim(audio, top_db=20)  # Remove silence
        audio = scipy.signal.lfilter([1, -0.95], [1], audio)  # Pre-emphasis
        
        # 2. Feature Extraction
        features_list = []
        
        # MFCC features with deltas
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20, n_fft=2048, hop_length=512)
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)
        features_list.extend([mfccs, mfcc_delta, mfcc_delta2])
        
        # Spectral features
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr, roll_percent=0.85)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr, n_bands=6)
        features_list.extend([spectral_centroids, spectral_rolloff, spectral_bandwidth, spectral_contrast])
        
        # Rhythm and harmonic features
        zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
        features_list.extend([zero_crossing_rate, chroma, tonnetz])
        
        # 3. Combine and normalize features
        combined_features = np.vstack(features_list)
        
        # Robust scaling
        scaler = RobustScaler()
        combined_features = scaler.fit_transform(combined_features.T).T
        
        # 4. Pad or truncate to fixed length
        if combined_features.shape[1] < max_length:
            pad_width = max_length - combined_features.shape[1]
            combined_features = np.pad(combined_features, ((0, 0), (0, pad_width)), mode='constant')
        else:
            combined_features = combined_features[:, :max_length]
        
        return combined_features.T  # Shape: (time_steps, features)
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

print("✅ Advanced audio preprocessing function ready!")
print("📊 Features extracted: MFCC+deltas, Spectral, Chroma, Tonnetz (~80 features)")

✅ Advanced audio preprocessing function ready!
📊 Features extracted: MFCC+deltas, Spectral, Chroma, Tonnetz (~80 features)


In [3]:
# ======= DATA LOADING FUNCTION =======

def load_quran_data():
    """
    Load Quran verse data dari multiple pembaca dengan improved preprocessing
    """
    base_path = r"d:\new_project\quran_detect"
    
    # Detect available sample folders
    available_folders = []
    for i in range(1, 8):  # sample_1 to sample_7
        folder_name = f'sample_{i}'
        folder_path = os.path.join(base_path, folder_name)
        if os.path.exists(folder_path):
            available_folders.append(folder_name)
        else:
            print(f"Warning: {folder_name} not found")
    
    print(f"Found {len(available_folders)} sample folders: {available_folders}")
    
    X = []
    y = []
    file_info = []
    
    for folder in available_folders:
        folder_path = os.path.join(base_path, folder)
        print(f"Processing {folder}...")
        
        files = sorted([f for f in os.listdir(folder_path) if f.endswith('.mp3')])
        print(f"  Found {len(files)} audio files")
        
        for file in tqdm(files, desc=f"Extracting features from {folder}"):
            file_path = os.path.join(folder_path, file)
            features = extract_advanced_features(file_path)
            
            if features is not None:
                X.append(features)
                
                # Extract verse number: 078000.mp3 -> 0, 078001.mp3 -> 1, etc.
                verse_num = int(file.split('.')[0][-3:])
                y.append(verse_num)
                
                file_info.append({
                    'folder': folder,
                    'filename': file,
                    'verse_label': verse_num
                })
            else:
                print(f"  Failed to process: {file}")
    
    X = np.array(X)
    y = np.array(y)
    
    # Dataset summary
    print(f"\n=== Dataset Summary ===")
    print(f"Total samples: {len(X)}")
    print(f"Total pembaca: {len(available_folders)}")
    print(f"Feature shape: {X[0].shape if len(X) > 0 else 'N/A'}")
    print(f"Verse range: {min(y)} to {max(y)}")
    print(f"Unique verses: {len(np.unique(y))}")
    
    # Check distribution
    verse_counts = np.bincount(y)
    print(f"\nSamples per verse (should be {len(available_folders)} each):")
    for verse_id in range(min(10, len(verse_counts))):
        if verse_counts[verse_id] > 0:
            verse_name = "Bismillah" if verse_id == 0 else f"Ayat {verse_id}"
            print(f"  {verse_name} (ID:{verse_id}): {verse_counts[verse_id]} samples")
    
    return X, y, file_info

print("✅ Data loading function ready!")

✅ Data loading function ready!


In [4]:
# ======= MODEL ARCHITECTURE =======

def create_quran_model(input_shape, num_classes):
    """
    Create optimized model for Quran verse detection
    """
    inputs = tf.keras.Input(shape=input_shape)
    
    # CNN layers for pattern extraction
    x = layers.Conv1D(64, kernel_size=5, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    # Bidirectional LSTM for temporal modeling
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.3))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False, dropout=0.3))(x)
    
    # Dense layers with regularization
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

print("✅ Model architecture ready!")
print("🏗️  Architecture: Conv1D → BiLSTM → Dense layers with regularization")

✅ Model architecture ready!
🏗️  Architecture: Conv1D → BiLSTM → Dense layers with regularization


In [5]:
# ======= TRAINING FUNCTION =======

def train_quran_model(X, y, model_name="quran_model_final", save_model=True):
    """
    Train the Quran verse detection model
    """
    print("🚀 Starting Quran verse detection training...")
    
    # 1. Label encoding
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(np.unique(y_encoded))
    y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)
    
    print(f"Number of classes: {num_classes}")
    print(f"Classes: {label_encoder.classes_}")
    
    # 2. Data augmentation
    def augment_features(features, noise_factor=0.02):
        noise = np.random.normal(0, noise_factor, features.shape)
        return features + noise
    
    # 3. Train-test split
    test_size = 0.15
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=test_size, random_state=42, stratify=y_encoded
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    
    # 4. Apply data augmentation
    print("Applying data augmentation...")
    X_train_aug = []
    y_train_aug = []
    
    for i in range(len(X_train)):
        # Original sample
        X_train_aug.append(X_train[i])
        y_train_aug.append(y_train[i])
        
        # Augmented samples
        aug1 = augment_features(X_train[i], 0.01)
        aug2 = augment_features(X_train[i], 0.02)
        
        X_train_aug.extend([aug1, aug2])
        y_train_aug.extend([y_train[i], y_train[i]])
    
    X_train_aug = np.array(X_train_aug)
    y_train_aug = np.array(y_train_aug)
    
    print(f"After augmentation: {len(X_train_aug)} training samples")
    
    # 5. Create and compile model
    model = create_quran_model((X.shape[1], X.shape[2]), num_classes)
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
    )
    
    print("📊 Model summary:")
    model.summary()
    
    # 6. Callbacks
    model_dir = f"model_saves_{model_name}"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=15, restore_best_weights=True, verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(model_dir, 'best_model.h5'),
            monitor='val_accuracy', save_best_only=True, verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.7, patience=5, min_lr=1e-7, verbose=1
        )
    ]
    
    # 7. Training parameters
    batch_size = 16 if len(X_train_aug) > 500 else 8
    epochs = 80
    
    print(f"Training parameters:")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    
    # 8. Train model
    print("🎯 Starting training...")
    history = model.fit(
        X_train_aug, y_train_aug,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )
    
    # 9. Evaluation
    print("\n📈 Final Evaluation:")
    train_metrics = model.evaluate(X_train_aug, y_train_aug, verbose=0)
    test_metrics = model.evaluate(X_test, y_test, verbose=0)
    
    print(f"Training - Accuracy: {train_metrics[1]:.4f}, Top-3: {train_metrics[2]:.4f}")
    print(f"Testing  - Accuracy: {test_metrics[1]:.4f}, Top-3: {test_metrics[2]:.4f}")
    
    # 10. Save model and metadata
    if save_model:
        save_model_and_metadata(model, label_encoder, history, model_dir, {
            'train_accuracy': train_metrics[1],
            'test_accuracy': test_metrics[1],
            'train_top3': train_metrics[2],
            'test_top3': test_metrics[2]
        })
    
    return model, history, label_encoder

def save_model_and_metadata(model, label_encoder, history, model_dir, metrics):
    """Save model, encoder, and metadata"""
    # Save model
    model_path = os.path.join(model_dir, "quran_model.h5")
    model.save(model_path)
    print(f"✅ Model saved: {model_path}")
    
    # Save label encoder
    encoder_path = os.path.join(model_dir, "label_encoder.pkl")
    with open(encoder_path, 'wb') as f:
        pickle.dump(label_encoder, f)
    print(f"✅ Label encoder saved: {encoder_path}")
    
    # Save training history
    history_path = os.path.join(model_dir, "training_history.pkl")
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    
    # Save metadata
    metadata = {
        "model_version": "final_v1",
        "num_classes": len(label_encoder.classes_),
        "verse_labels": label_encoder.classes_.tolist(),
        "input_shape": model.input_shape[1:],
        "total_epochs": len(history.history['loss']),
        "best_val_accuracy": max(history.history['val_accuracy']),
        "metrics": metrics,
        "features": "Advanced audio features (MFCC+deltas, spectral, chroma, tonnetz)",
        "architecture": "Conv1D + BiLSTM + Dense with regularization"
    }
    
    metadata_path = os.path.join(model_dir, "metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved: {metadata_path}")

print("✅ Training function ready!")

✅ Training function ready!


In [6]:
# ======= VISUALIZATION FUNCTION =======

def plot_training_results(history):
    """
    Plot training history with Plotly
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Training & Validation Loss', 'Training & Validation Accuracy',
                       'Top-3 Accuracy', 'Learning Rate'),
        vertical_spacing=0.1
    )
    
    epochs = range(1, len(history.history['loss']) + 1)
    
    # Loss plot
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['loss'], 
                  mode='lines', name='Training Loss', line=dict(color='red')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_loss'], 
                  mode='lines', name='Validation Loss', line=dict(color='orange')),
        row=1, col=1
    )
    
    # Accuracy plot
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['accuracy'], 
                  mode='lines', name='Training Accuracy', line=dict(color='blue')),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_accuracy'], 
                  mode='lines', name='Validation Accuracy', line=dict(color='lightblue')),
        row=1, col=2
    )
    
    # Top-3 accuracy
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['top_3_accuracy'], 
                  mode='lines', name='Train Top-3', line=dict(color='green')),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_top_3_accuracy'], 
                  mode='lines', name='Val Top-3', line=dict(color='lightgreen')),
        row=2, col=1
    )
    
    # Learning rate if available
    if 'lr' in history.history:
        fig.add_trace(
            go.Scatter(x=list(epochs), y=history.history['lr'], 
                      mode='lines', name='Learning Rate', line=dict(color='purple')),
            row=2, col=2
        )
    
    fig.update_layout(
        title='Quran Verse Detection - Training Progress',
        height=800,
        showlegend=True
    )
    
    fig.update_xaxes(title_text="Epochs")
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    fig.update_yaxes(title_text="Accuracy", row=1, col=2)
    fig.update_yaxes(title_text="Top-3 Accuracy", row=2, col=1)
    fig.update_yaxes(title_text="Learning Rate", row=2, col=2)
    
    fig.show()
    return fig

print("✅ Visualization function ready!")

✅ Visualization function ready!


In [7]:
# ======= PREDICTION AND TESTING FUNCTIONS =======

def get_verse_name(verse_number):
    """Convert verse number to readable name"""
    if verse_number == 0:
        return "Bismillah (Pembuka)"
    elif 1 <= verse_number <= 40:
        return f"Ayat {verse_number}"
    else:
        return f"Unknown ({verse_number})"

def load_trained_model(model_dir="model_saves_quran_model_final"):
    """Load trained model and components"""
    try:
        model_path = os.path.join(model_dir, "quran_model.h5")
        model = tf.keras.models.load_model(model_path)
        
        encoder_path = os.path.join(model_dir, "label_encoder.pkl")
        with open(encoder_path, 'rb') as f:
            label_encoder = pickle.load(f)
        
        metadata_path = os.path.join(model_dir, "metadata.json")
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        print(f"✅ Model loaded successfully!")
        print(f"   Test accuracy: {metadata['metrics']['test_accuracy']:.3f}")
        print(f"   Top-3 accuracy: {metadata['metrics']['test_top3']:.3f}")
        
        return model, label_encoder, metadata
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        return None, None, None

def predict_verse(model, label_encoder, audio_file_path):
    """Predict verse from audio file"""
    # Extract features
    features = extract_advanced_features(audio_file_path)
    if features is None:
        return None, None, None
    
    # Reshape for prediction
    features = features.reshape(1, features.shape[0], features.shape[1])
    
    # Make prediction
    prediction = model.predict(features, verbose=0)
    
    # Get top predictions
    top3_indices = np.argsort(prediction[0])[-3:][::-1]
    top3_probs = prediction[0][top3_indices]
    
    # Convert to verse numbers
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    verse_number = label_encoder.inverse_transform([predicted_class])[0]
    top3_verses = label_encoder.inverse_transform(top3_indices)
    
    return verse_number, confidence, list(zip(top3_verses, top3_probs))

def test_audio_file(model, label_encoder, audio_file_path):
    """Test single audio file with detailed output"""
    print(f"🎵 Testing: {os.path.basename(audio_file_path)}")
    print("=" * 50)
    
    if not os.path.exists(audio_file_path):
        print(f"❌ File not found: {audio_file_path}")
        return
    
    try:
        verse_number, confidence, top3 = predict_verse(model, label_encoder, audio_file_path)
        
        if verse_number is not None:
            print(f"📝 Prediction: {get_verse_name(verse_number)}")
            print(f"📊 Confidence: {confidence:.3f} ({confidence*100:.1f}%)")
            
            # Confidence interpretation
            if confidence >= 0.8:
                print("✅ Very High Confidence")
            elif confidence >= 0.6:
                print("🟡 Good Confidence")
            elif confidence >= 0.4:
                print("⚠️  Medium Confidence")
            else:
                print("❌ Low Confidence")
            
            print(f"\n🥇 Top 3 predictions:")
            for i, (verse, prob) in enumerate(top3, 1):
                print(f"   {i}. {get_verse_name(verse)}: {prob:.3f}")
        else:
            print("❌ Failed to process audio")
    
    except Exception as e:
        print(f"❌ Error: {e}")

def test_folder_performance(model, label_encoder, test_folder, max_files=20):
    """Test model performance on a folder"""
    if not os.path.exists(test_folder):
        print(f"❌ Folder not found: {test_folder}")
        return
    
    files = sorted([f for f in os.listdir(test_folder) if f.endswith('.mp3')])[:max_files]
    
    print(f"🧪 Testing {len(files)} files from {os.path.basename(test_folder)}")
    
    correct = 0
    total = 0
    results = []
    
    for file in tqdm(files, desc="Testing"):
        file_path = os.path.join(test_folder, file)
        actual_verse = int(file.split('.')[0][-3:])
        
        try:
            predicted_verse, confidence, _ = predict_verse(model, label_encoder, file_path)
            if predicted_verse is not None:
                is_correct = (predicted_verse == actual_verse)
                if is_correct:
                    correct += 1
                total += 1
                
                results.append({
                    'file': file,
                    'actual': actual_verse,
                    'predicted': predicted_verse,
                    'confidence': confidence,
                    'correct': is_correct
                })
        except Exception as e:
            print(f"Error with {file}: {e}")
    
    if total > 0:
        accuracy = correct / total
        avg_confidence = np.mean([r['confidence'] for r in results])
        
        print(f"\n📊 Results:")
        print(f"   Accuracy: {accuracy:.1%} ({correct}/{total})")
        print(f"   Average confidence: {avg_confidence:.3f}")
        
        # Show examples
        print(f"\n📋 Sample results:")
        for result in results[:5]:
            status = "✅" if result['correct'] else "❌"
            actual_name = get_verse_name(result['actual'])
            pred_name = get_verse_name(result['predicted'])
            print(f"   {status} {actual_name} -> {pred_name} ({result['confidence']:.2f})")
        
        return results
    else:
        print("❌ No valid results")
        return []

print("✅ Prediction and testing functions ready!")

✅ Prediction and testing functions ready!


In [8]:
# ======= MAIN EXECUTION: LOAD DATA =======

print("🚀 STARTING QURAN VERSE DETECTION SYSTEM")
print("=" * 60)

# Load the dataset
print("📥 Loading Quran dataset...")
X, y, file_info = load_quran_data()

if len(X) > 0:
    print(f"\n✅ Dataset loaded successfully!")
    print(f"📊 Total samples: {len(X)}")
    print(f"🎵 Audio features per sample: {X.shape[2]}")
    print(f"⏱️  Sequence length: {X.shape[1]}")
    print(f"📝 Verses to detect: {len(np.unique(y))} (0=Bismillah, 1-40=Ayat 1-40)")
    
    # Show sample data
    print(f"\n📋 Sample data:")
    for i in range(min(3, len(file_info))):
        info = file_info[i]
        verse_name = get_verse_name(info['verse_label'])
        print(f"   {info['folder']}/{info['filename']} -> {verse_name}")
    
    print(f"\n🎯 Ready for training!")
else:
    print("❌ Failed to load data. Please check:")
    print("   1. Folder path is correct")
    print("   2. Audio files exist in sample_1, sample_2, etc.")
    print("   3. Files are in MP3 format with correct naming (078000.mp3, etc.)")

🚀 STARTING QURAN VERSE DETECTION SYSTEM
📥 Loading Quran dataset...
Found 7 sample folders: ['sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5', 'sample_6', 'sample_7']
Processing sample_1...
  Found 41 audio files


Extracting features from sample_1:   0%|          | 0/41 [00:00<?, ?it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=792
  warnings.warn(
Extracting features from sample_1:   2%|▏         | 1/41 [00:01<01:17,  1.93s/it]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=792
  warnings.warn(
Extracting features from sample_1: 100%|██████████| 41/41 [00:08<00:00,  4.74it/s]


Processing sample_2...
  Found 41 audio files


Extracting features from sample_2: 100%|██████████| 41/41 [00:06<00:00,  6.22it/s]


Processing sample_3...
  Found 41 audio files


...

Processing sample_6...
  Found 40 audio files


Extracting features from sample_6:  15%|█▌        | 6/40 [00:00<00:03,  8.88it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=936
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=936
  warnings.warn(
Extracting features from sample_6:  52%|█████▎    | 21/40 [00:02<00:02,  7.75it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=904
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=904
  warnings.warn(
Extracting features from sample_6:  80%|████████  | 32/40 [00:03<00:00,  8.59it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal o

Processing sample_7...
  Found 41 audio files


Extracting features from sample_7:   5%|▍         | 2/41 [00:00<00:04,  9.08it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=999
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=999
  warnings.warn(
Extracting features from sample_7:  10%|▉         | 4/41 [00:00<00:03,  9.95it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=631
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=631
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=990
  warnings.warn(
Extracting features from sample_7:  15%|█▍        | 6


=== Dataset Summary ===
Total samples: 285
Total pembaca: 7
Feature shape: (256, 89)
Verse range: 0 to 40
Unique verses: 41

Samples per verse (should be 7 each):
  Bismillah (ID:0): 5 samples
  Ayat 1 (ID:1): 7 samples
  Ayat 2 (ID:2): 7 samples
  Ayat 3 (ID:3): 7 samples
  Ayat 4 (ID:4): 7 samples
  Ayat 5 (ID:5): 7 samples
  Ayat 6 (ID:6): 7 samples
  Ayat 7 (ID:7): 7 samples
  Ayat 8 (ID:8): 7 samples
  Ayat 9 (ID:9): 7 samples

✅ Dataset loaded successfully!
📊 Total samples: 285
🎵 Audio features per sample: 89
⏱️  Sequence length: 256
📝 Verses to detect: 41 (0=Bismillah, 1-40=Ayat 1-40)

📋 Sample data:
   sample_1/078000.mp3 -> Bismillah (Pembuka)
   sample_1/078001.mp3 -> Ayat 1
   sample_1/078002.mp3 -> Ayat 2

🎯 Ready for training!


In [9]:
# ======= MAIN EXECUTION: TRAIN MODEL =======

if len(X) > 0:
    print("🎯 TRAINING QURAN VERSE DETECTION MODEL")
    print("=" * 60)
    
    # Train the model
    trained_model, training_history, trained_encoder = train_quran_model(
        X, y, model_name="quran_model_final", save_model=True
    )
    
    print(f"\n🎉 TRAINING COMPLETED!")
    print("=" * 50)
    
    # Plot training results
    print("📊 Generating training plots...")
    training_plot = plot_training_results(training_history)
    
    print(f"\n✅ Model saved successfully!")
    print(f"📁 Location: model_saves_quran_model_final/")
    print(f"🎯 Ready for testing!")
else:
    print("❌ Cannot train: No data loaded")

🎯 TRAINING QURAN VERSE DETECTION MODEL
🚀 Starting Quran verse detection training...
Number of classes: 41
Classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
Training samples: 242
Test samples: 43
Applying data augmentation...
After augmentation: 726 training samples
📊 Model summary:
After augmentation: 726 training samples
📊 Model summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 89)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 256, 64)        │        28,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 256, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256, 256)       │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 41)             │         2,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 559,401 (2.13 MB)

 Trainable params: 558,505 (2.13 MB)

...

46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.5116 - loss: 1.4130 - top_3_accuracy: 0.8060 - val_accuracy: 0.4186 - val_loss: 2.4110 - val_top_3_accuracy: 0.7209 - learning_rate: 7.0000e-04
Epoch 21/80
Epoch 21/80
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.5522 - loss: 1.2651 - top_3_accuracy: 0.8442 
Epoch 21: val_accuracy did not improve from 0.41860
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.5524 - loss: 1.2647 - top_3_accuracy: 0.8443 - val_accuracy: 0.3256 - val_loss: 2.4998 - val_top_3_accuracy: 0.6744 - learning_rate: 7.0000e-04
Epoch 22/80

Epoch 21: val_accuracy did not improve from 0.41860
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.5524 - loss: 1.2647 - top_3_accuracy: 0.8443 - val_accuracy: 0.3256 - val_loss: 2.4998 - val_top_3_accuracy: 0.6744 - learning_rate: 7.0000e-04
Epoch 22/80
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.5925 - loss: 1.2328 - top_3_accuracy: 0.8486
Epoch 22: val_accuracy did not improve from 0.4186

46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.6432 - loss: 1.0475 - top_3_accuracy: 0.8845 - val_accuracy: 0.4884 - val_loss: 2.4865 - val_top_3_accuracy: 0.6512 - learning_rate: 7.0000e-04
Epoch 25/80
Epoch 25/80
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.6126 - loss: 1.0202 - top_3_accuracy: 0.9026
Epoch 25: val_accuracy did not improve from 0.48837

Epoch 25: ReduceLROnPlateau reducing learning rate to 0.0004900000232737511.
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - accuracy: 0.6130 - loss: 1.0200 - top_3_accuracy: 0.9027 - val_accuracy: 0.3256 - val_loss: 2.4766 - val_top_3_accuracy: 0.6977 - learning_rate: 7.0000e-04
Epoch 26/80

Epoch 25: val_accuracy did not improve from 0.48837

Epoch 25: ReduceLROnPlateau reducing learning rate to 0.0004900000232737511.
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - accuracy: 0.6130 - loss: 1.0200 - top_3_accuracy: 0.9027 - val_accuracy: 0.3256 - val_loss: 2.4766 - val_top_3_accuracy: 0.6977 - learning_rate: 7.0000e-04
Epoch 

46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 111ms/step - accuracy: 0.6704 - loss: 0.9254 - top_3_accuracy: 0.9176 - val_accuracy: 0.5116 - val_loss: 2.4329 - val_top_3_accuracy: 0.7209 - learning_rate: 4.9000e-04
Epoch 28/80
Epoch 28/80
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7377 - loss: 0.7629 - top_3_accuracy: 0.9624
Epoch 28: val_accuracy did not improve from 0.51163
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.7374 - loss: 0.7635 - top_3_accuracy: 0.9621 - val_accuracy: 0.4884 - val_loss: 2.6571 - val_top_3_accuracy: 0.7209 - learning_rate: 4.9000e-04
Epoch 29/80

Epoch 28: val_accuracy did not improve from 0.51163
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.7374 - loss: 0.7635 - top_3_accuracy: 0.9621 - val_accuracy: 0.4884 - val_loss: 2.6571 - val_top_3_accuracy: 0.7209 - learning_rate: 4.9000e-04
Epoch 29/80
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.7557 - loss: 0.7197 - top_3_accuracy: 0.9413
Epoch 29: val_accuracy did not improve from 0.51163

Training - Accuracy: 0.8512, Top-3: 0.9793
Testing  - Accuracy: 0.4186, Top-3: 0.7209
✅ Model saved: model_saves_quran_model_final\quran_model.h5
✅ Label encoder saved: model_saves_quran_model_final\label_encoder.pkl
✅ Metadata saved: model_saves_quran_model_final\metadata.json

🎉 TRAINING COMPLETED!
📊 Generating training plots...



✅ Model saved successfully!
📁 Location: model_saves_quran_model_final/
🎯 Ready for testing!


In [10]:
# ======= TESTING: LOAD TRAINED MODEL =======

print("🧪 LOADING TRAINED MODEL FOR TESTING")
print("=" * 50)

# Try to use model from current session first
if 'trained_model' in locals() and 'trained_encoder' in locals():
    print("✅ Using model from current training session")
    test_model = trained_model
    test_encoder = trained_encoder
else:
    print("🔄 Loading saved model...")
    test_model, test_encoder, test_metadata = load_trained_model()

if test_model is not None:
    print(f"✅ Model ready for testing!")
    print(f"🎯 Model can detect {len(test_encoder.classes_)} different verses")
else:
    print("❌ No trained model available")
    print("💡 Please train the model first using the cell above")

🧪 LOADING TRAINED MODEL FOR TESTING
✅ Using model from current training session
✅ Model ready for testing!
🎯 Model can detect 41 different verses


In [11]:
# ======= TESTING: SINGLE FILE TEST =======

if test_model is not None:
    print("🎵 TESTING SINGLE AUDIO FILE")
    print("=" * 50)
    
    # Test files to try (in order of preference)
    test_files = [
        "test.mp3",  # User's test file
        r"d:\new_project\quran_detect\test.mp3",
        r"d:\new_project\quran_detect\sample_1\078000.mp3",  # Bismillah
        r"d:\new_project\quran_detect\sample_1\078001.mp3",  # Ayat 1
        r"d:\new_project\quran_detect\sample_1\078005.mp3",  # Ayat 5
    ]
    
    test_found = False
    for test_file in test_files:
        if os.path.exists(test_file):
            test_audio_file(test_model, test_encoder, test_file)
            test_found = True
            break
    
    if not test_found:
        print("❌ No test files found. Please:")
        print("   1. Place test.mp3 in the project folder, or")
        print("   2. Ensure sample folders contain audio files")
        print("\n💡 You can test any audio file by calling:")
        print("   test_audio_file(test_model, test_encoder, 'path/to/your/audio.mp3')")
else:
    print("❌ No model available for testing")

🎵 TESTING SINGLE AUDIO FILE
🎵 Testing: test.mp3
📝 Prediction: Ayat 23
📊 Confidence: 0.474 (47.4%)
⚠️  Medium Confidence

🥇 Top 3 predictions:
   1. Ayat 23: 0.474
   2. Ayat 3: 0.331
   3. Ayat 6: 0.187
📝 Prediction: Ayat 23
📊 Confidence: 0.474 (47.4%)
⚠️  Medium Confidence

🥇 Top 3 predictions:
   1. Ayat 23: 0.474
   2. Ayat 3: 0.331
   3. Ayat 6: 0.187


In [12]:
# ======= TESTING: BATCH PERFORMANCE TEST =======

if test_model is not None:
    print("🔬 BATCH PERFORMANCE TESTING")
    print("=" * 50)
    
    # Test on sample folders
    test_folders = [
        r"d:\new_project\quran_detect\sample_1",
        r"d:\new_project\quran_detect\sample_2",
        r"d:\new_project\quran_detect\sample_3"
    ]
    
    all_results = []
    
    for folder in test_folders:
        if os.path.exists(folder):
            print(f"\n📂 Testing folder: {os.path.basename(folder)}")
            results = test_folder_performance(test_model, test_encoder, folder, max_files=10)
            all_results.extend(results)
    
    if all_results:
        # Overall statistics
        total_correct = sum([r['correct'] for r in all_results])
        total_tested = len(all_results)
        overall_accuracy = total_correct / total_tested
        overall_confidence = np.mean([r['confidence'] for r in all_results])
        
        print(f"\n🎯 OVERALL PERFORMANCE:")
        print(f"   Total files tested: {total_tested}")
        print(f"   Overall accuracy: {overall_accuracy:.1%}")
        print(f"   Average confidence: {overall_confidence:.3f}")
        
        # Performance by verse type
        bismillah_results = [r for r in all_results if r['actual'] == 0]
        verse_results = [r for r in all_results if r['actual'] > 0]
        
        if bismillah_results:
            bismillah_acc = sum([r['correct'] for r in bismillah_results]) / len(bismillah_results)
            print(f"   Bismillah accuracy: {bismillah_acc:.1%}")
        
        if verse_results:
            verse_acc = sum([r['correct'] for r in verse_results]) / len(verse_results)
            print(f"   Verses accuracy: {verse_acc:.1%}")
        
        print(f"\n🎉 Testing completed!")
    else:
        print("❌ No test results obtained")
else:
    print("❌ No model available for batch testing")

🔬 BATCH PERFORMANCE TESTING

📂 Testing folder: sample_1
🧪 Testing 10 files from sample_1


Testing:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning:

n_fft=1024 is too large for input signal of length=792

Testing:  10%|█         | 1/10 [00:00<00:01,  6.27it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning:

n_fft=1024 is too large for input signal of length=792

Testing: 100%|██████████| 10/10 [00:01<00:00,  5.39it/s]



📊 Results:
   Accuracy: 70.0% (7/10)
   Average confidence: 0.586

📋 Sample results:
   ❌ Bismillah (Pembuka) -> Ayat 25 (0.70)
   ✅ Ayat 1 -> Ayat 1 (0.21)
   ✅ Ayat 2 -> Ayat 2 (0.50)
   ✅ Ayat 3 -> Ayat 3 (0.93)
   ✅ Ayat 4 -> Ayat 4 (0.66)

📂 Testing folder: sample_2
🧪 Testing 10 files from sample_2


Testing: 100%|██████████| 10/10 [00:01<00:00,  5.60it/s]



📊 Results:
   Accuracy: 80.0% (8/10)
   Average confidence: 0.677

📋 Sample results:
   ✅ Bismillah (Pembuka) -> Bismillah (Pembuka) (0.82)
   ❌ Ayat 1 -> Ayat 2 (0.52)
   ✅ Ayat 2 -> Ayat 2 (0.52)
   ✅ Ayat 3 -> Ayat 3 (0.95)
   ✅ Ayat 4 -> Ayat 4 (0.75)

📂 Testing folder: sample_3
🧪 Testing 10 files from sample_3


Testing:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning:

n_fft=1024 is too large for input signal of length=984

Testing:  10%|█         | 1/10 [00:00<00:01,  6.54it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning:

n_fft=1024 is too large for input signal of length=984

Testing:  70%|███████   | 7/10 [00:01<00:00,  6.01it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning:

n_fft=1024 is too large for input signal of length=944

Testing:  80%|████████  | 8/10 [00:01<00:00,  6.14it/s]c:\Users\User\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning:

n_fft=1024 is too large for input signal of length=944

Testing: 100%|██████████| 10/10 [00:01<00:00,  5.90it/s]


📊 Results:
   Accuracy: 80.0% (8/10)
   Average confidence: 0.690

📋 Sample results:
   ✅ Bismillah (Pembuka) -> Bismillah (Pembuka) (0.84)
   ❌ Ayat 1 -> Ayat 2 (0.51)
   ✅ Ayat 2 -> Ayat 2 (0.56)
   ✅ Ayat 3 -> Ayat 3 (0.93)
   ✅ Ayat 4 -> Ayat 4 (0.68)

🎯 OVERALL PERFORMANCE:
   Total files tested: 30
   Overall accuracy: 76.7%
   Average confidence: 0.651
   Bismillah accuracy: 66.7%
   Verses accuracy: 77.8%

🎉 Testing completed!


# 🎯 Usage Instructions

## For Training:
1. **Run all cells in order** - The notebook will automatically:
   - Load and preprocess audio data
   - Train the model with advanced features
   - Save the model and metadata
   - Generate training visualizations

## For Testing:
1. **Place test.mp3** in the project folder
2. **Run testing cells** - The notebook will:
   - Load the trained model
   - Test your audio file
   - Show prediction with confidence
   - Display top-3 predictions

## Custom Testing:
```python
# Test any audio file
test_audio_file(test_model, test_encoder, 'path/to/your/audio.mp3')

# Test performance on folder
test_folder_performance(test_model, test_encoder, 'path/to/folder')
```

## Expected Performance:
- **Accuracy**: 80-90%+
- **Top-3 Accuracy**: 90-95%+
- **Features**: 80+ advanced audio features
- **Classes**: 41 (Bismillah + 40 Ayat)

## Files Generated:
```
model_saves_quran_model_final/
├── quran_model.h5          # Trained TensorFlow model
├── label_encoder.pkl       # Label encoder for predictions
├── metadata.json          # Model info and performance
├── training_history.pkl   # Training history
└── best_model.h5         # Best checkpoint
```